# Preparation

In [1]:
import math
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from trafficSimulator import *
import matplotlib.pyplot as plt

pygame 2.0.1 (SDL 2.0.14, Python 3.6.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Simulation setup

### Simulation params setup

In [2]:
all_routes = [[0, 3], [0, 1], [2, 3], [2, 1]]

v_max = 10
s0 = 2
T = 1
b_max = 2
a_max = 1

vehicle_specs = {'v_max': v_max, # Desired speed
                 's0': s0, # Safe bumper-to-bumper distance
                 'T': T, # Time gap
                 'b_max': b_max, # Deceleartion
                 'a_max': a_max, # Acceleration
                }

vehicle_rate = 120
fast_track_factor, slow_track_factor = 1, 0.1
stop_distance = 40
fast_wait_time = 1
slow_wait_time = 5
vehicle_limit = 20

df = pd.DataFrame(columns=['Vehicle_label', 'Road_order', 
                           'Total_time', 'Leading_vehicles',
                           'Stopped_time', 'Stop_while_front'])

# vehicle_preferences = dict(zip(range(vehicle_limit), 
#                                [[0.25, 0.25, 0.25, 0.25]]*vehicle_limit))

vehicle_preferences = dict(zip(range(vehicle_limit), np.random.choice(len(all_routes), size=vehicle_limit)))

### Topology setup

In [3]:
def parallel_line(current_coordinates, distance):
    """
    Returns the coordinate sets of lines that are parallel to the given line
    with the speicified distance
    
    Input:
        current_coordinates: two set of x and y coordinates that mark the start
                             and end of the given line
        distance: the distance between the given line and the parallel lines to
                  to be output
    """
    
    x1, y1 = current_coordinates[0]
    x2, y2 = current_coordinates[1]
    k = (y1-y2)/(x1-x2)
    b = y1 - k*x1
    
    cross_x = -b/k
    hypo = math.sqrt(b**2+cross_x**2)
    
    if k>0:
        new_coord_1 = ((x2+distance*abs(b)/hypo, y2-distance*abs(cross_x)/hypo),
                       (x1+distance*abs(b)/hypo, y1-distance*abs(cross_x)/hypo))

        new_coord_2 = ((x2-distance*abs(b)/hypo, y2+distance*abs(cross_x)/hypo),
                       (x1-distance*abs(b)/hypo, y1+distance*abs(cross_x)/hypo))
        
    else:
        new_coord_1 = ((x2+distance*abs(b)/hypo, y2+distance*abs(cross_x)/hypo),
                       (x1+distance*abs(b)/hypo, y1+distance*abs(cross_x)/hypo))
    
        new_coord_2 = ((x2-distance*abs(b)/hypo, y2-distance*abs(cross_x)/hypo),
                       (x1-distance*abs(b)/hypo, y1-distance*abs(cross_x)/hypo))
    
    return new_coord_1, new_coord_2

In [4]:
left, right = -80*math.sqrt(3), 80*math.sqrt(3)
bottom, top = -80, 80

left_bottom_outbound = ((left+2, 4), (-5, top-2),
                        slow_track_factor, stop_distance, slow_wait_time)
bottom_right_outbound = ((5, top-2), (right-2, 4),
                         fast_track_factor, stop_distance, fast_wait_time)

left_top_outbound = ((left+2, -4), (-5, bottom+2),
                     fast_track_factor, stop_distance, fast_wait_time)
top_right_outbound = ((5, bottom+2), (right-2, -4),
                      slow_track_factor, stop_distance, slow_wait_time)

bottom_left_inbound = parallel_line(current_coordinates=left_bottom_outbound, 
                                    distance=2.5)[0]
right_bottom_inbound = parallel_line(current_coordinates=bottom_right_outbound, 
                                     distance=2.5)[1]

top_left_inbound = parallel_line(current_coordinates=left_top_outbound, 
                                 distance=2.5)[0]
right_top_inbound = parallel_line(current_coordinates=top_right_outbound, 
                                  distance=2.5)[1]

## For the connections of the topology, we can either represent with concrete lines
## or just as a dot (as shown below). Either case, the vehicles will not actually go 
## pass these routes; they are more for demonstration purposes

connection_top_bottom = ((-1.25, bottom+2), (-1.25, top-2))
connection_bottom_top = ((1.25, top-2), (1.25, bottom+2))

# Or just show a dot:

# connection_top_bottom = ((-1.25, -2), (-1.25, 2))
# connection_bottom_top = ((1.25, 2), (1.25, -2))

### Utility params setup

In [5]:
# Smaller delta encourages more exploration
delta = 0.5

# Penalty proportion for delay caused
alpha = 0

# Interpolation factor between money and time
gamma_mean, gamma_var = 10, 5

# Level of risk aversion
eta_mean, eta_var = 1, 0.5

## Some functions needed

### For running simulation

In [6]:
 def run_simulation(round_number, all_routes, vehicle_limit, df, vehicle_preferences,
                    vehicle_rate, vehicle_specs):
    """
    Executes the simulation and returns certain data of the simulation.
    
    Input:
        round_number (integer): the number of the current simulation
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    global left_bottom_outbound
    global bottom_right_outbound
    global left_top_outbound
    global top_right_outbound
    global connection_top_bottom
    global connection_bottom_top
    global bottom_left_inbound
    global right_bottom_inbound
    global top_left_inbound
    global right_top_inbound
    
    records = df.copy()
    sim = Simulation({
        'round_number': round_number,
        'all_routes': all_routes, # All possible (and reasonable) routes
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'records': records, # Table that will capture the needed vehicle-related info
        'vehicle_preferences': vehicle_preferences
        # A dictionary that stores the vehicles' labels as keys and their list of probabilities
        # of choosing each possible route as values
        # For the initial few rounds all vehicles choose all routes with equal weights.
        # For more complicated topologies some other algorithms will be needed as all possible
        # routes are not as explicit as in here.
        })

    sim.create_roads([
        ## Key routes
        left_bottom_outbound, # Road #0
        bottom_right_outbound, # Road #1

        left_top_outbound, # Road #2
        top_right_outbound, # Road #3

        connection_top_bottom, # Road #4
        connection_bottom_top, # Road #5

#         bottom_left_inbound, # Road #6
#         right_bottom_inbound, # Road #7

#         top_left_inbound, # Road #8
#         right_top_inbound, # Road #9

        ## Curved corners
        # Note: in the simulation, the vehicles will not actually go pass these
        # routes; they are more for aesthetic purposes

        # Outbound corners
        *curve_road(left_bottom_outbound[1], 
                    (bottom_right_outbound[0][0], bottom_right_outbound[0][1]+0.01), 
                    (0, top), 16), # Outbound bottom corner

        *curve_road(left_top_outbound[1], 
                    (top_right_outbound[0][0], top_right_outbound[0][1]+0.01), 
                    (0, bottom), 16), # Outbound top corner

        *curve_road(left_bottom_outbound[0], 
                    (left_top_outbound[0][0]+0.01, left_top_outbound[0][1]), 
                    (left, 0), 16), # Outbound left corner

        *curve_road(bottom_right_outbound[1], 
                    (top_right_outbound[1][0]+0.01, top_right_outbound[1][1]), 
                    (right, 0), 16), # Outbound right corner

#         # Inbound corners
#         *curve_road(right_bottom_inbound[1], 
#                     (bottom_left_inbound[0][0], bottom_left_inbound[0][1]+0.01), 
#                     (0, top), 16), # Inbound bottom corner

#         *curve_road(right_top_inbound[1], 
#                     (top_left_inbound[0][0], top_left_inbound[0][1]+0.01), 
#                     (0, bottom), 16), # Inbound top corner

#         *curve_road(bottom_left_inbound[1], 
#                     (top_left_inbound[1][0]+0.01, top_left_inbound[1][1]), 
#                     (left+2.5, 0), 16), # Inbound left corner

#         *curve_road(right_bottom_inbound[0], 
#                     (right_top_inbound[0][0]+0.01, right_top_inbound[0][1]), 
#                     (right-2.5, 0), 16), # Inbound right corner
    ])


    sim.create_gen({
        'vehicle_rate': vehicle_rate, # Rate of generating new vehicles
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'vehicles': vehicle_specs
        })

    # Start simulation
    win = Window(sim)
    win.zoom = 3
    new_records = win.run(steps_per_update=5)
    
    return new_records

In [7]:
# a = [
#         ## Key routes
#         left_bottom_outbound, # Road #0
#         bottom_right_outbound, # Road #1

#         left_top_outbound, # Road #2
#         top_right_outbound]

In [8]:
# for i in a:
#     start, end, slow_factor, stop_distance, wait_time = i

In [9]:
# stop_distance, wait_time

### For simulation result analysis

In [10]:
# a = record_df.copy()

# a.loc[(a.Vehicle_label==46) & (a.Round_number==1), 'Total_time'] += 1

In [11]:
def utility_summary(record_df):
    """
    Returns a pivot table that summarizes the utility related features of each vehicle.
    
    Input:
        record_df (pandas DataFrame): records the simulation data of each vehicle
    
    """
    
    utility_tmp = record_df[['Vehicle_label', 'Road_order', 'Total_time', 'Round_number']]

    utility_tmp['Caused_delay'] = 0.0
    utility_tmp.Total_time = pd.to_numeric(utility_tmp.Total_time)

    utility_tmp = utility_tmp.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)

    for idx, row in record_df.iterrows():
        current_l = row['Vehicle_label']
        stop_times = row['Stopped_time']
        lead_v = row['Leading_vehicles']
        route = row['Road_order']
        round_num = row['Round_number']

        # For each vehicle's stop time, find its leading vehicle of the corresponding road segment
        # and assign the stop time to that vehicle as the time of delay that it causes
        for i in range(len(stop_times)):
            if stop_times[i] > 0:
                utility_tmp.loc[(utility_tmp.Vehicle_label==lead_v[i]) &\
                                (utility_tmp.Round_number==round_num), 'Caused_delay'] += stop_times[i]
#                 utility_tmp.at[lead_v[i], 'Caused_delay'] += stop_times[i]

        # Record the label of the routes taken
        for r in all_routes:
            if route == r:
                utility_tmp.at[idx, 'Road_order'] = all_routes.index(r)

    utility_tmp['Count'] = 1
    utility_tmp = pd.pivot_table(utility_tmp, 
                                 index=['Vehicle_label', 'Road_order'], 
                                 values=['Total_time', 'Caused_delay', 'Count'], 
                                 aggfunc={'Total_time': np.mean, # Average time taken for the past rounds
                                          'Caused_delay': np.mean, # Average delay caused for the
                                                                   # past rounds
                                          'Count': 'count'} # Count how many times the vehicle has taken
                                                            # the same routes
                                ).reset_index(drop=False)
    return utility_tmp

In [12]:
def compute_utility(explored_times,
                    time=0, 
                    caused_delay=0, 
                    alpha=0, 
                    gamma=0, 
                    eta=0, 
                    delta=0.1):
    """
    Returns the computed utility value
    
    Input:
        explored_times (integer): the number of times that the vehicle has taken the
                                  same route
        time (float): the amount of time taken to reach the destination
        caused_delay (float): the amount of delay that the vehicle has caused
        alpha (float): the penalty level
        gamma (float): the interpolation between money and time spent
        eta (float): the level of risk aversion
        delta (float): a parameter of the Upper Confidence Bound equation that controls
                       extent of encouraging exploration
    
    """
    
    if explored_times > 0:
        
        if alpha != 0:
            u_penalty = -np.log(caused_delay*alpha) 
        else:
            u_penalty = 0
            
        if eta == 1:
            u_time = -np.log(time)
        else:
            u_time = - (time**(1-eta)-1) / (1-eta)
            
        u_total = gamma * u_time + u_penalty
    else:
        u_total = 0
        
    if explored_times < 5:
        u_total += np.sqrt(2*np.log(1/delta)/explored_times)
    return u_total

# def compute_probability(utility_list):
#     """
#     Returns a list of probabilities of taking each route (using softmax)
    
#     """
#     prob_list = [np.exp(u) for u in utility_list]
#     return [p/sum(prob_list) for p in prob_list]

In [13]:
# # np.exp(-np.log(40)), np.exp(-np.log(30))
# -np.log(40), -np.log(30)

In [14]:
def update_utility_df(record_df, df, utility_tmp, round_number=1):
    """
    Returns an updated dataframe that stores utility data of all vehicles
    
    """
    
    global all_routes
    utility_df = df.copy()
    
    for idx, row in utility_df.iterrows():
        i = row['Vehicle_label']
        utility_l = row['Utilities'][:]
        routes = row['Routes_taken'][:]
        gamma = row['Gamma']
        eta = row['Eta']

        for j in range(len(all_routes)):
            record = utility_tmp.loc[(utility_tmp.Vehicle_label==i) & (utility_tmp.Road_order==j)]
            if len(record) == 0:
                utility_l[j] = compute_utility(explored_times=0.01, delta=delta) # Exploration time is not integer here
                                                                                 # because utility will be infinite if
                                                                                 # it is 0 and it will cause problems in prob
                                                                                 # calculation
            else:
                utility_l[j] = compute_utility(explored_times=record['Count'].values[0], 
                                               time=record['Total_time'].values[0], 
                                               caused_delay=record['Caused_delay'].values[0], 
                                               alpha=alpha, 
                                               gamma=gamma, 
                                               eta=eta, 
                                               delta=delta)
        
        # Append the route chosen for the last round
        new_route_list = record_df.loc[(record_df.Vehicle_label==i) &
                                       (record_df.Round_number==round_number)]['Road_order'].values[0]
        new_route = all_routes.index(new_route_list)
        routes.append(new_route)
    
        # Update dataframe
        utility_df.at[idx, 'Utilities'] = utility_l
        utility_df.at[idx, 'Choice'] = utility_l.index(max(utility_l))
        utility_df.at[idx, 'Routes_taken'] = routes
    
    return utility_df

In [15]:
a = [1, 3, 2]
a.index(max(a))

1

# Official start: 1st round

In [16]:
%%time
# Run the 1st round
record_df1 = run_simulation(1, all_routes, vehicle_limit, df, vehicle_preferences,
                             vehicle_rate, vehicle_specs)

Wall time: 38.8 s


In [17]:
record_df1

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front
0,0,"[2, 3]",106.35,"[-999, -999]","[0, 0]","[0.05, 0]"
1,1,"[2, 3]",117.7,"[0, 0]","[0, 0]","[0, 0]"
2,2,"[0, 3]",127.65,"[-999, 1]","[0, 0.05]","[17.700000000000117, 0]"
3,3,"[2, 1]",60.7,"[1, -999]","[0.1, 0]","[0.05, 0]"
4,4,"[0, 1]",70.2,"[2, 3]","[15.80000000000009, 0]","[0.05, 0]"
5,5,"[0, 1]",72.9,"[4, 4]","[15.15000000000008, 0]","[0, 0]"
6,6,"[0, 3]",141.9,"[5, 10]","[0.2, 0]","[0, 0]"
7,7,"[0, 3]",147.7,"[6, 6]","[0.2, 0]","[0, 0]"
8,8,"[0, 1]",86.8,"[7, 13]","[19.200000000000138, 0]","[0, 0]"
9,9,"[0, 3]",160,"[8, 17]","[17.700000000000117, 0]","[0, 0]"


## Simulation Analysis of the 1st round

In [18]:
record_df1['Round_number'] = 1

record_df = record_df1.copy()
record_df.head(10)

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front,Round_number
0,0,"[2, 3]",106.35,"[-999, -999]","[0, 0]","[0.05, 0]",1
1,1,"[2, 3]",117.7,"[0, 0]","[0, 0]","[0, 0]",1
2,2,"[0, 3]",127.65,"[-999, 1]","[0, 0.05]","[17.700000000000117, 0]",1
3,3,"[2, 1]",60.7,"[1, -999]","[0.1, 0]","[0.05, 0]",1
4,4,"[0, 1]",70.2,"[2, 3]","[15.80000000000009, 0]","[0.05, 0]",1
5,5,"[0, 1]",72.9,"[4, 4]","[15.15000000000008, 0]","[0, 0]",1
6,6,"[0, 3]",141.9,"[5, 10]","[0.2, 0]","[0, 0]",1
7,7,"[0, 3]",147.7,"[6, 6]","[0.2, 0]","[0, 0]",1
8,8,"[0, 1]",86.8,"[7, 13]","[19.200000000000138, 0]","[0, 0]",1
9,9,"[0, 3]",160,"[8, 17]","[17.700000000000117, 0]","[0, 0]",1


### Set up dataframe

In [19]:
utility_df = pd.DataFrame(columns=['Vehicle_label', 'Utilities', 'Choice'])

utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
utility_df['Routes_taken'] = [[]] * len(utility_df)
utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
utility_df['Choice'] = -99

In [20]:
etas = []
gammas = []

for i in range(len(utility_df)):
    eta = np.random.normal(eta_mean, eta_var, 1)[0]
    while eta < 0:
        eta = np.random.normal(eta_mean, eta_var, 1)[0]
    etas.append(eta)
    
for i in range(len(utility_df)):
    gamma = np.random.normal(gamma_mean, gamma_var, 1)[0]
    while gamma < 0:
        gamma = np.random.normal(gamma_mean, gamma_var, 1)[0]
    gammas.append(gamma)

utility_df['Gamma'] = gammas
utility_df['Eta'] = etas

utility_df = utility_df.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)            
utility_df.head(5)

,Vehicle_label,Utilities,Choice,Routes_taken,Gamma,Eta
0,0,"[0, 0, 0, 0]",-99,[],0.529789,1.723622
1,1,"[0, 0, 0, 0]",-99,[],7.317145,0.732115
2,2,"[0, 0, 0, 0]",-99,[],13.316744,1.334845
3,3,"[0, 0, 0, 0]",-99,[],8.183519,1.106529
4,4,"[0, 0, 0, 0]",-99,[],14.760149,1.082987


## Summary of the 1st round

In [21]:
utility_tmp = utility_summary(record_df)
utility_df = update_utility_df(record_df=record_df,
                               df=utility_df, 
                               utility_tmp=utility_tmp)

display(utility_tmp.head(10))
display(utility_df.head(10))

,Vehicle_label,Road_order,Caused_delay,Count,Total_time
0,0,2,0.00,1,106.35
1,1,2,0.15,1,117.70
2,2,0,15.80,1,127.65
3,3,3,0.00,1,60.70
4,4,1,15.15,1,70.20
5,5,1,0.20,1,72.90
6,6,0,0.20,1,141.90
7,7,0,19.25,1,147.70
8,8,1,17.70,1,86.80
9,9,0,15.85,1,160.00


,Vehicle_label,Utilities,Choice,Routes_taken,Gamma,Eta
0,0,"[11.774100225154747, 11.774100225154747, 1.554...",0,[2],0.529789,1.723622
1,1,"[11.774100225154747, 11.774100225154747, 9.568...",0,[2],7.317145,0.732115
2,2,"[12.49396872050077, 11.774100225154747, 11.774...",0,[0],13.316744,1.334845
3,3,"[11.774100225154747, 11.774100225154747, 11.77...",0,[3],8.183519,1.106529
4,4,"[11.774100225154747, 15.341707604984132, 11.77...",1,[1],14.760149,1.082987
5,5,"[11.774100225154747, 25.688156057131643, 11.77...",1,[1],14.711110,0.053144
6,6,"[16.519421202846225, 11.774100225154747, 11.77...",0,[0],13.034734,0.682442
7,7,"[13.869690795712048, 11.774100225154747, 11.77...",0,[0],13.125968,1.067578
8,8,"[11.774100225154747, 12.032115159185452, 11.77...",1,[1],7.082283,0.199158
9,9,"[9.767004811946231, 11.774100225154747, 11.774...",1,[0],5.908609,0.293179


In [22]:
# # Export
# record_df.to_csv("Saved_data/Simulation_records/Round_1.csv", index=False)
# utility_df.to_csv("Saved_data/Utility_data/Round_1.csv", index=False)

# Run the remaining rounds
Utilities, probabilities are updated after each round and fed into the next round

In [ ]:
%%time

import time
time_l = []
begin = time.time()

# Updated probabilities for the 2nd round:
vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Choice.to_list()))

rounds = 50

for i in range(2, rounds+1):
    record_tmp = run_simulation(i, all_routes, 
                                vehicle_limit, 
                                df, vehicle_preferences,
                                vehicle_rate, vehicle_specs)
    record_tmp['Round_number'] = i
    
    record_df = pd.concat([record_df, record_tmp], ignore_index=True)
    
    utility_tmp = utility_summary(record_df)
    utility_df = update_utility_df(record_df=record_df,
                                   df=utility_df, 
                                   utility_tmp=utility_tmp,
                                   round_number=i)
    
    vehicle_preferences = dict(zip(range(vehicle_limit), 
                                   utility_df.Choice.to_list()))
    
    # Export
#     record_df.to_csv(f"Saved_data/Simulation_records/Round_{i}.csv", index=False)
#     utility_df.to_csv(f"Saved_data/Utility_data/Round_{i}.csv", index=False)
    
    time_l.append(round(time.time()-begin-sum(time_l), 2))

In [ ]:
# utility_tmp.to_csv(f"Saved_data/Simulation_records/Summary.csv", index=False)

In [ ]:
# sum(time_l)/60

In [ ]:
record_df.tail(20)

# Visualization

In [ ]:
# utility_df = pd.read_csv("Saved_data/Utility_data/Round_20.csv")
# record_df = pd.read_csv("Saved_data/Simulation_records/Round_20.csv")
# rounds = 20
# vehicle_limit = 25

In [ ]:
# display(utility_df.head(10))
# display(record_df)

In [ ]:
# import ast
# a = utility_df.Routes_taken[0]
# b = ast.literal_eval(a)
# b[0]

In [ ]:
# utility_df.Routes_taken.to_list()[1]

In [ ]:
# gamma mean = 10
import ast
fig, ax = plt.subplots(figsize=(10, 6))

for i in range(vehicle_limit):
#     route_list = ast.literal_eval(utility_df.Routes_taken.to_list()[i])
    route_list = utility_df.Routes_taken.to_list()[i]
    ax.scatter(range(rounds), route_list, alpha=0.5, color='g')
    
# ax.show()

# Others

In [ ]:
# from scipy.io import loadmat
# eta_data = loadmat('data_mapEtas.mat')

# eta_data["mapEtaVals"]